In [ ]:
!pip install onnxruntime-genai
!pip install onnxruntime
!pip install olive
!pip install huggingface-hub

In [ ]:
!huggingface-cli login --token <TOKEN>

In [6]:
import olive.workflows

olive.workflows.run("olive.json")

[2024-04-10 15:01:07,980] [INFO] [run.py:243:run] Loading Olive module configuration from: C:\Users\nakersha\AppData\Local\miniconda3\envs\assistant\Lib\site-packages\olive\olive_config.json
[2024-04-10 15:01:07,989] [INFO] [run.py:249:run] Loading run configuration from: olive.json
[2024-04-10 15:01:08,021] [INFO] [config.py:184:validate_evaluate_input_model] No evaluator is specified, skip to evaluate model
[2024-04-10 15:01:08,044] [DEBUG] [accelerator.py:245:normalize_accelerators] The accelerator device and execution providers are specified, skipping deduce.
[2024-04-10 15:01:08,050] [DEBUG] [accelerator.py:275:normalize_accelerators] Supported execution providers for device cpu: ['CPUExecutionProvider']
[2024-04-10 15:01:08,053] [DEBUG] [accelerator.py:302:create_accelerators] Initial accelerators and execution providers: {'CPU': ['CPUExecutionProvider']}
[2024-04-10 15:01:08,056] [INFO] [accelerator.py:324:create_accelerators] Running workflow on accelerator specs: cpu-cpu
[2024

{AcceleratorSpec(accelerator_type=<Device.CPU: 'cpu'>, execution_provider='CPUExecutionProvider', vender=None, version=None, memory=None, num_cores=None): <olive.engine.footprint.Footprint at 0x242948c3cd0>}

In [ ]:
import onnxruntime_genai as og
import time

In [ ]:
print("Loading model...")
app_started_timestamp = time.time()

model = og.Model(f'..\..\microsoft\onnxruntime-genai\examples\python\example-models\phi2-int4-cpu-acc4')
model_loaded_timestamp  = time.time()

print("Model loaded in {:.2f} seconds".format(model_loaded_timestamp - app_started_timestamp))



In [ ]:
print("Loading tokenizer...")
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

print("Tokenizer created")

system_prompt = "You are a helpful assistant. Answer in one sentence."
text = "What is Dilithium?"

input_tokens = tokenizer.encode(system_prompt + text)

prompt_length = len(input_tokens)

In [ ]:
started_timestamp = time.time()

print("Creating generator ...")
params = og.GeneratorParams(model)
params.set_search_options({"do_sample": False, "max_length": 2028, "min_length": 0, "top_p": 0.9, "top_k": 40, "temperature": 1.0, "repetition_penalty": 1.0})
params.input_ids = input_tokens
generator = og.Generator(model, params)
print("Generator created")

first = True
new_tokens = []

while not generator.is_done():
    generator.compute_logits()
    generator.generate_next_token()
    if first:
        first_token_timestamp = time.time()
        first = False

    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end="")
    new_tokens.append(new_token)

print()
run_time = time.time() - started_timestamp
print(f"Prompt tokens: {len(input_tokens)}, New tokens: {len(new_tokens)}, Time to first: {(first_token_timestamp - started_timestamp):.2f}s, New tokens per second: {len(new_tokens)/run_time:.2f} tps")


## Download and build llama.cpp
git clone https://github.com/ggerganov/llama.cpp.git
cd llama.cpp
cmake -S . -B build/ -D CMAKE_BUILD_TYPE=Release
cmake --build build/ --config Release

## Download gguf phi-2 model
git clone https://huggingface.co/TheBloke/phi-2-GGUF

In [ ]:
# Compare with llama.cpp.

! ..\..\ggerganov\llama.cpp\build\bin\Release\main -m ..\..\thebloke\phi-2-GGUF\phi-2.Q4_K_M.gguf --prompt "You are a helpful assistant. Answer in one sentence. What is Dilithium?"
